In [1]:
!pip3 install lxml
import pandas as pd
import numpy as np
import json
from datetime import date,timedelta

In [2]:
pd.set_option('display.max_rows',None)

In [3]:
base_nuggets_href ="https://www.basketball-reference.com/teams/DEN/{season}/gamelog"
base_lakers_href ="https://www.basketball-reference.com/teams/LAL/{season}/gamelog"
years_to_go_back = 4
nuggets_seasons =[]
lakers_seasons =[]
for i in range(years_to_go_back,-1,-1):
    nuggets_seasons.append(base_nuggets_href.format(season=(date.today().year)-i))
    lakers_seasons.append(base_lakers_href.format(season=(date.today().year)-i))


print(nuggets_seasons)
print(lakers_seasons)

['https://www.basketball-reference.com/teams/DEN/2016/gamelog', 'https://www.basketball-reference.com/teams/DEN/2017/gamelog', 'https://www.basketball-reference.com/teams/DEN/2018/gamelog', 'https://www.basketball-reference.com/teams/DEN/2019/gamelog', 'https://www.basketball-reference.com/teams/DEN/2020/gamelog']
['https://www.basketball-reference.com/teams/LAL/2016/gamelog', 'https://www.basketball-reference.com/teams/LAL/2017/gamelog', 'https://www.basketball-reference.com/teams/LAL/2018/gamelog', 'https://www.basketball-reference.com/teams/LAL/2019/gamelog', 'https://www.basketball-reference.com/teams/LAL/2020/gamelog']


Create Data Frame from team game log

In [4]:
df = pd.read_html('https://www.basketball-reference.com/teams/LAL/2020/gamelog/',skiprows=(22,23,44,45,66,67,88,89))[0]

Creates copy of dataframe then undoes multiIndex for Dataframe

In [5]:
df1 = df.copy(deep=True)
df1.columns = [" ".join(i) for i in df1.columns]

Gets only advanced we are interested in and renames unnammed columns

In [6]:
data = df1[['Unnamed: 6_level_0 Tm','Unnamed: 7_level_0 Opp','Unnamed: 3_level_0 Unnamed: 3_level_1','Unnamed: 4_level_0 Opp','Team FG%','Opponent FG%','Team 3P%','Opponent 3P%','Team AST','Team TOV','Opponent AST','Opponent TOV','Team FGA','Team FTA','Opponent FGA','Opponent FTA','Team ORB','Opponent ORB']]

In [7]:
data.rename({'Unnamed: 6_level_0 Tm':'Team Score'},axis=1,inplace=True)
data.rename({'Unnamed: 7_level_0 Opp':'Opp Score'},axis=1,inplace=True)
data.rename({'Unnamed: 3_level_0 Unnamed: 3_level_1':'Home/Away'},axis=1,inplace=True)
data.rename({'Unnamed: 4_level_0 Opp':'Opponent'},axis=1,inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Creates copy of new dataframe and adds new advance stats columns that will be calculated with data in dataframe

In [8]:
advance = data.copy(deep=True)

In [9]:
advance['ORtg'] = 0
advance['DRtg'] = 0
advance['Ast_Tov_Team'] = 0
advance['Ast_Tov_Opp'] = 0
advance['Possession'] = 0

In [10]:
ast_tov_team = advance['Team AST']/advance['Team TOV']
ast_tov_opp = advance['Opponent AST']/advance['Opponent TOV']

In [11]:
advance['Ast_Tov_Team'] = ast_tov_team
advance['Ast_Tov_Opp'] = ast_tov_opp

In [12]:
possession = .5*(advance['Team FGA'] + (.475*advance['Team FTA']) - advance['Team ORB'] + advance['Team TOV']) + .5 * (advance['Team FGA'] + (.475*advance['Opponent FTA']) - advance['Opponent ORB'] + advance['Opponent TOV'])
advance['Possession'] = possession

In [13]:
ortg = 100 * (advance['Team Score']/advance['Possession'])
drtg = 100 * (advance['Opp Score']/advance['Possession'])

In [14]:
advance['ORtg'] = ortg
advance['DRtg'] = drtg

Adds Team number and Opp numbers

In [15]:
list2 = []
for index,row in advance.iterrows():
    if row['Home/Away'] == '@':
         list2.append(0)
    else:
        list2.append(1)
advance['Home/Away'] = list2

In [16]:
advance['Team'] = 16
advance['Opponent_num'] = 0

In [17]:
with open('abr_numbers.json') as f:
    data1 = json.load(f)
team_name = list(data1.keys())
team_number = list(data1.values())
f.close()

In [18]:
list_ = []
for index,row in advance.iterrows():
    if row.Opponent in data1:
        list_.append(data1[row.Opponent])
advance['Opponent_num'] = list_

Places data in csv 

In [19]:
advance.to_csv('lakers_advance_2020.csv',index=False)